# Estrattore degli appalti

In [124]:
# Import
import pandas as pd
import numpy as np
import matplotlib
import logging
import urllib
import io
import csv
import time
import os
from lxml import etree

# Constraints
APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetAppaltiL190.xsd"
INDICE_APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetIndiceAppaltiL190.xsd"

In [125]:
def get_node(node):
    try:
        return node.text
    except (AttributeError, IndexError):
        return np.nan

def get_node2(node,index):
    try:
        return node[index].text
    except (AttributeError, IndexError):
        return np.nan

def get_date(node):
    try:
        data_inizio= node[0].text
    except (AttributeError, IndexError):
        data_inizio= np.nan

    try:
        data_fine= node[1].text
    except (AttributeError, IndexError):
        data_fine= np.nan
    
    return data_inizio, data_fine

def appalti_xml2csv(doc,counter_so_far, tid, rid, arid):
    csv_file = open(("../data/appalti2/%d.csv" % counter_so_far), 'w')
    csv_writer = csv.writer(csv_file, delimiter=';', quoting=csv.QUOTE_ALL)
    root = doc.getroot()
    anno_rif = get_node(root[0][5])
    
    lines=[]
    for lotto in root[1]:
        cig = get_node(lotto[0])
        codice_fiscale_prop, denominazione  = get_node(lotto[1][0]), get_node(lotto[1][1])
        oggetto, scelta_contraente = get_node(lotto[2]), get_node(lotto[3])
        importo_aggiudicazione, importo_somme_liquidate = get_node(lotto[6]), get_node(lotto[8])
        data_inizio, data_fine = get_date(lotto[7]) 

        
        lines.append([tid, anno_rif,cig, codice_fiscale_prop, denominazione, oggetto, scelta_contraente, importo_aggiudicazione, importo_somme_liquidate, data_inizio, data_fine])
        
        partecipanti, aggiundicatari = lotto[4], lotto[5]
        partecipanti.set('tid', "%d"%tid)
        aggiundicatari.set('tid', "%d"%tid)
        tid+=1
        
    csv_writer.writerows(lines)
    csv_file.close()
    del lines

    for element in root.iter("raggruppamento","aggiudicatarioRaggruppamento"):
        if element.tag=='raggruppamento':
            element.set('rid', "%d"%rid)
            rid+=1
        elif element.tag=='aggiudicatarioRaggruppamento':
            element.set('arid', "%d"%arid)
            arid+=1
        else:
            raise NameError("Ops! Iterando su raggruppamenti ho trovato un errore.")

    csv_file = open(("../data/fornitori/%d.csv" % counter_so_far), 'w')
    csv_writer = csv.writer(csv_file, delimiter=';', quoting=csv.QUOTE_ALL)
    lines=[]
    for element in root.iter("membro", "partecipante","aggiudicatario"):
        parent=element.getparent()
        granparent = parent.getparent()
            
        if parent.tag=='raggruppamento':
            #linea è tid:granparent, rid:parent_rid, arid=none ... il resto uguale
            lines.append([granparent.attrib['tid'],parent.attrib['rid'],np.NaN, get_node2(element,0), get_node2(element,1), get_node2(element,2), 0, 0])
        elif parent.tag=='aggiudicatarioRaggruppamento':
            #identico al precedente ma speculare su rid e arid
            lines.append([granparent.attrib['tid'], np.NaN ,parent.attrib['arid'], get_node2(element,0), get_node2(element,1), get_node2(element,2),0,0])
        elif element.tag == "partecipante":
            # tid viene preso dal parent, rid e arid settati a Nan                                                                                                                 
            lines.append([parent.attrib['tid'], np.NaN ,np.NaN, get_node2(element,0), get_node2(element,1), np.NaN, 1, 0])
        elif element.tag == "aggiudicatario":
            # tid viene preso dal parent, rid e arid settati a Nan                                                                                                                 
            lines.append([parent.attrib['tid'], np.NaN ,np.NaN, get_node2(element,0), get_node2(element,1), np.NaN, 0,1])
        else:
            raise NameError("Ops! Iterando su raggruppamenti ho trovato un errore.")
    csv_writer.writerows(lines)
    del lines
    csv_file.close()
    return tid, rid, arid

In [126]:
!head -2 ../data/comunicazioni.csv

"codiceFiscale";"dataUltimoTentativoAccessoUrl";"esitoUltimoTentativoAccessoUrl";"identificativoPEC";"ragioneSociale";"url";"ext"
"80003710789";"2015-04-22T18:42:06.137+0000";"fallito";"opec275.20140526151617.31733.02.1.48@pec.aruba.it";"AMMINISTRAZIONE PROVINCIALE DI COSENZA";"http://servizi.provincia.cs.it/legge190/2013/indice_dataset.xml";"xml"


In [127]:
# Carica
comunicazioni = pd.read_csv('../data/comunicazioni.csv', sep=';', quoting=csv.QUOTE_ALL)
comunicazioni.drop((comunicazioni[comunicazioni['esitoUltimoTentativoAccessoUrl'] == 'fallito']).index, inplace=True)

In [128]:
# Ricavare lo schema degli appalti e indice
appalti_xmlschema = etree.XMLSchema(etree.parse(APPALTI_XSD))
indice_xmlschema = etree.XMLSchema(etree.parse(INDICE_APPALTI_XSD))

# UnComment for test
#comunicazioni.drop(comunicazioni.loc[50:].index, inplace=True);

In [129]:
logger = logging.getLogger('e02-appalti-xml2.csv')
logger.setLevel(logging.DEBUG)

if not len(logger.handlers):
    # create a file handler
    #%rm e02-appalti-xml2csv_test.log
    handler = logging.FileHandler('e02-appalti-xml2csv.log')
    handler.setLevel(logging.DEBUG)

    # create a logging format
    formatter = logging.Formatter('%(asctime)s;%(name)s;%(levelname)s;%(message)s')
    handler.setFormatter(formatter)

    # add the handlers to the logger
    logger.addHandler(handler)

In [130]:
#logger.removeHandler(logger.handlers[0])
#logger.handlers

In [132]:
#%%timeit

logger.info("=== Start ===")
url=['../data/test/acqueblspa_legge190_2014.xml', '../data/test/Cittadella-contratti-2014.xml', '../data/test/dataset1.xml','../data/test/sangimignano.xml', '../data/test/test.xml']
counter_so_far, tid, rid, arid=0,0,0,0

# Per ogni url 
# ... Prova a scaricare l'xml se va in errore scrivi in log l'errore
# ... se lo schema è appalti trasforma in csv e scrivi in log
# ... se lo schema è indice scrivi in log
# ... altrimenti logga xml schema sconosciuto
for u in comunicazioni.loc[counter_so_far:, 'url'].values:
#for u in url:
    time.sleep( 1 )
    try:
        doc = etree.parse(u)
        etree.strip_tags(doc,etree.Comment)
        if appalti_xmlschema.validate(doc):
            tid, rid, arid=appalti_xml2csv(doc, counter_so_far, tid, rid, arid)
            logger.info(";%d;appalti;fatto" %(counter_so_far))
        elif indice_xmlschema.validate(doc):
            logger.info(";%d;indice;saltato" %(counter_so_far))    
        else:
            logger.info(";%d;schema_errato;saltato" %(counter_so_far))
    except (urllib.error.URLError, urllib.error.HTTPError, OSError, etree.XMLSyntaxError) as e:
        logger.info(";%d;errore:%s;saltato" %(counter_so_far, type(e).__name__))
 
    counter_so_far +=1

KeyboardInterrupt: 